In [70]:
from pydantic_ai import Agent
from pydantic_graph import BaseNode, GraphRunContext, End, Graph
from pydantic_ai.providers.google_gla import GoogleGLAProvider
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai.format_as_xml import format_as_xml
from dotenv import load_dotenv
import os
from pydantic import Field, BaseModel
from typing import  List, Dict, Optional, Union, Tuple
from enum import Enum
from pptx.util import Inches
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor

from dataclasses import dataclass
from IPython.display import Image, display  
import time
import requests
load_dotenv()

google_api_key=os.getenv('google_api_key')
pse=os.getenv('pse')
llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=google_api_key))

In [81]:


@dataclass
class State:
    instruction: str
    presentation_style: str
    research: str
    presentation_plan: List[Dict]
    presentation: Dict
    presentation_config: Dict



class ChartType(str, Enum):
    COLUMN_CLUSTERED = "column_clustered"
    LINE = "line"
    PIE = "pie"
    BAR = "bar"
    SCATTER = "scatter"
class Position(BaseModel):
    left: float
    top: float
    width: float
    height: float

class Font_color(BaseModel):
    red: int
    green: int
    blue: int

class TextFormat(BaseModel):
# Prevent additional properties
        
    font_size: Optional[float] = Field(default=11.0, description="Font size in points")
    font_color: Optional[Font_color] = Field(default=Font_color(red=0, green=0, blue=0), description="RGB color tuple")
    is_bold: Optional[bool] = Field(default=False)
    is_italic: Optional[bool] = Field(default=False)
    alignment: Optional[str] = Field(default="LEFT")  # Use string instead of PP_ALIGN

class BulletPoint(BaseModel):

        
    text: str
    level: int = Field(default=0, ge=0, le=8, description="Bullet point level (0-8)")
    format: Optional[TextFormat] = Field(default_factory=TextFormat)

class ChartData(BaseModel):

    type: ChartType
    title: str
    categories: List[str]
    series: List[Dict[str, List[float]]]
    position: Position = Field(
        default=Position(left=2, top=2, width=6, height=4.5),
        description="(left, top, width, height) in inches"
    )

class TextBox(BaseModel):

    text: str
    position: Position = Field(
        default=Position(left=1, top=1, width=8, height=5),
        description="(left, top, width, height) in inches"
    )
    format: Optional[TextFormat] = Field(default_factory=TextFormat)

class SlideContent(BaseModel):

        
    title: Optional[str] = None
    bullet_points: Optional[List[BulletPoint]] = None
    text_boxes: Optional[List[TextBox]] = None
    # chart: Optional[ChartData] = None
    image_url: Optional[str] = None
    image_position: Optional[Position] = Field(
        default=None,
        description="(left, top, width, height) in inches"
    )



class PresentationConfig(BaseModel):

    title: str
    slides: List[SlideContent]=None
    subtitle: Optional[str] = None
    output_path: Optional[str] = None
    theme: Optional[str] = None  # Path to theme file
    slide_width: float = Field(default=10.0)
    slide_height: float = Field(default=7.5)




def google_image_search(query:str):
  """Search for images using Google Custom Search API
  args: query
  return: image url
  """
  # Define the API endpoint for Google Custom Search
  url = "https://www.googleapis.com/customsearch/v1"

  params = {
      "q": query,
      "cx": pse,
      "key": google_api_key,
      "searchType": "image",  # Search for images
      "num": 1  # Number of results to fetch
  }

  # Make the request to the Google Custom Search API
  response = requests.get(url, params=params)
  data = response.json()

  # Check if the response contains image results
  if 'items' in data:
      # Extract the first image result
      image_url = data['items'][0]['link']
      return image_url
  
text_extraction_agent=Agent(llm, system_prompt='extract the text from the reaserch based on the instructions')

class step_execution_node(BaseNode[State]):
    async def run(self, ctx: GraphRunContext[State])-> End:
        for page in ctx.state.presentation.keys():
            time.sleep(2)
            if ctx.state.presentation[page].get('image_title'):
                ctx.state.presentation[page]['image_url']=google_image_search(ctx.state.presentation[page]['image_title'])
            if ctx.state.presentation[page].get('text_to_extract'):
                result=await text_extraction_agent.run(f'extract the text from the research: {ctx.state.research} based on the instructions: {ctx.state.presentation[page]['text_to_extract']}')
                ctx.state.presentation[page]['text_content']=result.data
        
        return End(ctx.state.presentation_config)
    

class clean_up_node(BaseNode[State]):
    async def run(self, ctx: GraphRunContext[State])-> step_execution_node:
        page_number=1
        for task in ctx.state.presentation_plan.tasks:
            ctx.state.presentation[f'page_{page_number}']={'title':task.text_data.Text_title, 'text_to_extract':task.text_data.Text_to_extract,'image_title':task.image_data.image_title if task.image_data else None,'image_url':task.image_data.image_url if task.image_data else None}
            page_number+=1
        return step_execution_node()
    
pptx_agent=Agent(llm, result_type=PresentationConfig, system_prompt='generate a base presentation config based on the presentation plan')
class pptx_presentation(BaseNode[State]):
    async def run(self, ctx: GraphRunContext[State])-> clean_up_node:
        result=await pptx_agent.run(f'generate a base presentation config based on the presentation plan: {ctx.state.presentation_plan}')
        ctx.state.presentation_config=result.data
        return clean_up_node()
    



@dataclass
class image:
    image_title: Optional[str] = Field(description='the image title')
    image_url: Optional[str] = None

@dataclass
class text:
    Text_to_extract: str = Field(description=' a description of the text to extract from the research')
    Text_title: str = Field(description='the text title')
    Text_content: str = None

@dataclass
class steps:
    image_data: Optional[image] = Field(description='the image data, optional')
    text_data: text = Field(description='the text data')

@dataclass
class Presentation_plan:
    tasks: List[steps] = Field(description='the steps to complete')


presentation_plan_agent=Agent(llm, result_type=Presentation_plan, system_prompt='generate a presentation plan based on the research and instruction (if any), choose wether to use images or not for each step')
        
class Presentation_plan_node(BaseNode[State]):
    async def run(self, ctx: GraphRunContext[State])->step_execution_node:
        prompt=f'generate a presentation plan based on the research: {ctx.state.research} and instruction: {ctx.state.instruction} (if any), choose wether to use images or not for each step, make sure that the sources URLs are included in the presentation'
        result=await presentation_plan_agent.run(prompt)
        ctx.state.presentation_plan=result.data
        return step_execution_node()

In [82]:
class Presentation_gen:
    def __init__(self):
        self.graph=Graph(nodes=[Presentation_plan_node, step_execution_node, clean_up_node, pptx_presentation])
        self.state=State(research='', presentation_plan=[], presentation={}, presentation_config={}, presentation_style='', instruction='')

    async def chat(self,research:str, presentation_style:Optional[str]='None', instruction:Optional[str]='None'):
        """Chat with the presentation generator,
        Args:
            research (str): The research to generate a presentation for
            presentation_style (str): The style of the presentation
            instruction (str): The instruction for the presentation
        Returns:
            str: The response from the presentation generator
        """
        self.state.research=research
        self.state.presentation_style=presentation_style
        self.state.instruction=instruction
        response=await self.graph.run(Presentation_plan_node(),state=self.state)
        return response.output


    def display_graph(self):
        """Display the graph of the deep research engine
        Returns:
            Image: The image of the graph
        """
        image=self.graph.mermaid_image()
        return display(Image(image))

In [77]:
search='# Blind Snakes of the Bahamas\n\nBlind snakes, belonging to the family Typhlopidae, are small, burrowing snakes with a worm-like appearance. Several species are found in the Bahamas, including the Bahamian slender blind snake (Cubatyphlops biminiensis) and the Brown Blind Snake (Typhlops lumbricalis).\n\n## Species\n\n*   **Bahamian Slender Blind Snake (Cubatyphlops biminiensis):** This species is endemic to the Bahamas. Its conservation status is listed as Least Concern.\n*   **Brown Blind Snake (Typhlops lumbricalis):** This species is found on most of the larger islands of the Bahamas, including the Abacos, Andros, the Berry Islands, Eleuthera, Cat Island, Long Island, and throughout the Exuma chain. On Bimini, they are commonly found in rotting logs and under rocks.\n\n## Characteristics\n\nBlind snakes are typically small and slender, resembling worms. They are adapted for burrowing, with smooth scales and reduced eyes.\n\n## Habitat and Distribution\n\nBahamian blind snakes inhabit various environments, including coppice dry forest habitat. They are often found under rocks, in rotting logs, and in the soil.\n\n## Conservation\n\nWhile some blind snake species are listed as Least Concern, they still face threats such as habitat loss and degradation due to human activities like urbanization and agriculture. Conservation efforts are important to ensure the survival of these unique reptiles.\n\n##Interesting facts\n\nIt is really hard to see these SMALL snakes in the Bahamas. Take one look at them, and you will notice they look more like small worms than the other snakes that live in the Bahamas.\n\n## References\n\n*   Animalia.bio: [https://animalia.bio/bahamian-slender-blind-snake](https://animalia.bio/bahamian-slender-blind-snake)\n*   Wikipedia: [https://en.wikipedia.org/wiki/Bahamian_slender_blind_snake](https://en.wikipedia.org/wiki/Bahamian_slender_blind_snake)\n*   Hummingbirdsplus.org: [https://www.hummingbirdsplus.org/nature-blog-network/the-5-types-of-snakes-found-on-the-bahamas/](https://www.hummingbirdsplus.org/nature-blog-network/the-5-types-of-snakes-found-on-the-bahamas/)\n*   Biminisharklab.com: [https://www.biminisharklab.com/fauna-of-bimini-1/bahamian-brown-blind-snake](https://www.biminisharklab.com/fauna-of-bimini-1/bahamian-brown-blind-snake)\n*   journals.ku.edu: [https://journals.ku.edu/reptilesandamphibians/article/download/15667/14618](https://journals.ku.edu/reptilesandamphibians/article/download/15667/14618)\n*   Allenpress.com: [https://meridian.allenpress.com/herpetologica/article/67/2/194/32795/Taxonomy-of-the-Blind-Snakes-Associated-with](https://meridian.allenpress.com/herpetologica/article/67/2/194/32795/Taxonomy-of-the-Blind-Snakes-Associated-with)\n*   Birdwatchinghq.com: [https://birdwatchinghq.com/snakes-of-the-Bahamas/](https://birdwatchinghq.com/snakes-of-the-Bahamas/)\n*   Photoguides.org: [https://photoguides.org/snakes-in-the-bahamas/](https://photoguides.org/snakes-in-the-bahamas/)\n*   Wildexplained.com: [https://wildexplained.com/animal-encyclopedia/the-western-blind-snake-uncovering-the-mystery-of-the-elusive-reptile/](https://wildexplained.com/animal-encyclopedia/the-western-blind-snake-uncovering-the-mystery-of-the-elusive-reptile/)\n*   Wildexplained.com: [https://wildexplained.com/animal-encyclopedia/discovering-the-mysterious-blind-snake/](https://wildexplained.com/animal-encyclopedia/discovering-the-mysterious-blind-snake/)\n\n'

In [83]:
pres=Presentation_gen()

In [84]:
res= await pres.chat(search)

UnexpectedModelBehavior: Received empty model response

In [85]:
pres.state.presentation_config

PresentationConfig(title='Blind Snakes of the Bahamas', slides=[SlideContent(title='Title: Blind Snakes of the Bahamas', bullet_points=None, text_boxes=None, image_url=None, image_position=None)], subtitle=None, output_path=None, theme=None, slide_width=10.0, slide_height=7.5)

In [142]:
pres.state.presentation_plan.tasks

[steps(image_data=image(image_title='Title Slide', image_url='https://slidebazaar.com/wp-content/uploads/2023/02/Title-Slide-For-PowerPoint-Presentation.jpg'), text_data=text(Text_to_extract='Introduction to Blind Snakes in the Bahamas', Text_title='Blind Snakes of the Bahamas', Text_content='Blind snakes, belonging to the family Typhlopidae, are small, burrowing snakes with a worm-like appearance. Several species are found in the Bahamas, including the Bahamian slender blind snake (Cubatyphlops biminiensis) and the Brown Blind Snake (Typhlops lumbricalis).\n')),
 steps(image_data=image(image_title='Species', image_url='https://m.media-amazon.com/images/M/MV5BMjEzOTkxNTcyM15BMl5BanBnXkFtZTcwOTk0MjI3NA@@._V1_.jpg'), text_data=text(Text_to_extract='Details of Bahamian Slender Blind Snake and Brown Blind Snake', Text_title='Species of Blind Snakes', Text_content='**Bahamian Slender Blind Snake (Cubatyphlops biminiensis):** This species is endemic to the Bahamas. Its conservation status is

In [107]:

class ChartType(str, Enum):
    COLUMN_CLUSTERED = "column_clustered"
    LINE = "line"
    PIE = "pie"
    BAR = "bar"
    SCATTER = "scatter"
@dataclass
class Position:
    left: float
    top: float
    width: float
    height: float

@dataclass
class Font_color:
    red: int
    green: int
    blue: int

@dataclass
class TextFormat:
# Prevent additional properties
        
    font_size: Optional[float] = Field(default=11.0, description="Font size in points")
    font_color: Optional[Font_color] = Field(default=Font_color(red=0, green=0, blue=0), description="RGB color tuple")
    is_bold: Optional[bool] = Field(default=False)
    is_italic: Optional[bool] = Field(default=False)
    alignment: Optional[str] = Field(default="LEFT")  # Use string instead of PP_ALIGN
@dataclass
class BulletPoint:
    class Config:
        extra = "forbid"
        
    text: str
    level: int = Field(default=0, ge=0, le=8, description="Bullet point level (0-8)")
    format: Optional[TextFormat] = Field(default_factory=TextFormat)
@dataclass
class ChartData:

    type: ChartType
    title: str
    categories: List[str]
    series: List[Dict[str, List[float]]]
    position: Position = Field(
        default=Position(left=2, top=2, width=6, height=4.5),
        description="(left, top, width, height) in inches"
    )
@dataclass
class TextBox:

    text: str
    position: Position = Field(
        default=Position(left=1, top=1, width=8, height=5),
        description="(left, top, width, height) in inches"
    )
    format: Optional[TextFormat] = Field(default_factory=TextFormat)
@dataclass
class SlideContent:

        
    title: Optional[str] = None
    bullet_points: Optional[List[BulletPoint]] = None
    text_boxes: Optional[List[TextBox]] = None
    # chart: Optional[ChartData] = None
    image_url: Optional[str] = Field()
    image_position: Optional[Position] = None

class Presentation_Config(BaseModel):
        
    title: str
    slides: List[SlideContent]=[]
    subtitle: Optional[str] = None
    output_path: Optional[str] = None
    theme: Optional[str] = None  # Path to theme file
    slide_width: float = Field(default=10.0)
    slide_height: float = Field(default=7.5)

In [120]:
pptx_agent2=Agent(llm, result_type=Presentation_Config, system_prompt='generate a base presentation config based on the presentation plan')


In [123]:
result=await pptx_agent2.run(f'generate a base presentation config with fake slides with fake text_boxes with fake text and fake images_url')

In [124]:
result.data

Presentation_Config(title='My Presentation', slides=[SlideContent(title='Slide 1', bullet_points=None, text_boxes=None, image_url=None, image_position=None), SlideContent(title='Slide 2', bullet_points=None, text_boxes=None, image_url=None, image_position=None)], subtitle=None, output_path=None, theme=None, slide_width=10.0, slide_height=7.5)

In [126]:
pages=pres.state.presentation
pages['page_2']


{'title': 'Species',
 'text': '**Bahamian Slender Blind Snake (Cubatyphlops biminiensis):** This species is endemic to the Bahamas. Its conservation status is listed as Least Concern.\n\n**Brown Blind Snake (Typhlops lumbricalis):** This species is found on most of the larger islands of the Bahamas, including the Abacos, Andros, the Berry Islands, Eleuthera, Cat Island, Long Island, and throughout the Exuma chain. On Bimini, they are commonly found in rotting logs and under rocks.\n',
 'image_title': 'Species Overview',
 'image_url': 'https://www.researchgate.net/publication/350641349/figure/fig1/AS:11431281172835652@1688664006197/Overview-of-the-approach-to-determine-species-ranges-classify-citizen-science.png'}

In [114]:
instruction=format_as_xml(SlideContent())

In [115]:
instruction

'<examples>\n  <title>null</title>\n  <bullet_points>null</bullet_points>\n  <text_boxes>null</text_boxes>\n  <image_url>null</image_url>\n  <image_position>null</image_position>\n</examples>'

In [118]:
slide_agent=Agent(llm, result_type=SlideContent, system_prompt='generate presentation slides based on the page schema, title and content are separate text_boxes')
result=await slide_agent.run(f'generate a presentation slide based on the page schema:{format_as_xml(pages['page_2'])} based on the instruction: {instruction}')


In [119]:
result.data

SlideContent(title='Species', bullet_points=None, text_boxes=[TextBox(text='**Bahamian Slender Blind Snake (Cubatyphlops biminiensis):** This species is endemic to the Bahamas. Its conservation status is listed as Least Concern.\n\n**Brown Blind Snake (Typhlops lumbricalis):** This species is found on most of the larger islands of the Bahamas, including the Abacos, Andros, the Berry Islands, Eleuthera, Cat Island, Long Island, and throughout the Exuma chain. On Bimini, they are commonly found in rotting logs and under rocks.', position=Position(left=1.0, top=3.0, width=5.0, height=6.0), format=TextFormat(font_size=FieldInfo(annotation=Union[float, NoneType], required=False, default=11.0, description='Font size in points'), font_color=FieldInfo(annotation=Union[Font_color, NoneType], required=False, default=Font_color(red=0, green=0, blue=0), description='RGB color tuple'), is_bold=FieldInfo(annotation=Union[bool, NoneType], required=False, default=False), is_italic=FieldInfo(annotation